In [1]:
# %pip install pyspark

In [2]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import broadcast, substring, col, avg, monotonically_increasing_id, lit, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [3]:
sc = SparkContext(appName="Simple App")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/25 08:35:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_schema=StructType([
    StructField('tourney_id', StringType(), True),
    StructField('tourney_name', StringType(), True),
    StructField('surface', StringType(), True),
    StructField('draw_size', IntegerType(), True),
    StructField('tourney_level', StringType(), True),
    StructField('tourney_date', IntegerType(), True),
    StructField('match_num', IntegerType(), True),
    StructField('winner_id', IntegerType(), True),
    StructField('winner_seed', IntegerType(), True),
    StructField('winner_entry', IntegerType(), True),
    StructField('winner_name', StringType(), True),
    StructField('winner_hand', StringType(), True),
    StructField('winner_ht', IntegerType(), True),
    StructField('winner_ioc', StringType(), True),
    StructField('winner_age', FloatType(), True),
    StructField('loser_id', IntegerType(), True),
    StructField('loser_seed', IntegerType(), True),
    StructField('loser_entry', IntegerType(), True),
    StructField('loser_name', StringType(), True),
    StructField('loser_hand', StringType(), True),
    StructField('loser_ht', IntegerType(), True),
    StructField('loser_ioc', StringType(), True),
    StructField('loser_age', FloatType(), True),
    StructField('score', StringType(), True),
    StructField('best_of', IntegerType(), True),
    StructField('round', StringType(), True),
    StructField('minutes', IntegerType(), True),
    StructField('w_ace', IntegerType(), True),
    StructField('w_df', IntegerType(), True),
    StructField('w_svpt', IntegerType(), True),
    StructField('w_1stIn', IntegerType(), True),
    StructField('w_1stWon', IntegerType(), True),
    StructField('w_2ndWon', IntegerType(), True),
    StructField('w_SvGms', IntegerType(), True),
    StructField('w_bpSaved', IntegerType(), True),
    StructField('w_bpFaced', IntegerType(), True),
    StructField('l_ace', IntegerType(), True),
    StructField('l_df', IntegerType(), True),
    StructField('l_svpt', IntegerType(), True),
    StructField('l_1stIn', IntegerType(), True),
    StructField('l_1stWon', IntegerType(), True),
    StructField('l_2ndWon', IntegerType(), True),
    StructField('l_SvGms', IntegerType(), True),
    StructField('l_bpSaved', IntegerType(), True),
    StructField('l_bpFaced', IntegerType(), True),
    StructField('winner_rank', IntegerType(), True),
    StructField('winner_rank_points', IntegerType(), True),
    StructField('loser_rank', IntegerType(), True),
    StructField('loser_rank_points', IntegerType(), True)
])

df = spark.read.schema(df_schema).csv('../files/atp/atp_matches_*', header=True)

In [5]:
# Carlos Alcaraz ID 207989
alcaraz_df = df.select(
    col('tourney_name'), 
    col('surface'),
    col('draw_size'),
    col('tourney_level'), 
    col('tourney_date'),
    col('winner_id'),
    col('winner_seed'),
    col('winner_name'),
    col('winner_hand'),
    col('winner_ioc'),
    col('winner_age'),
    col('loser_id'),
    col('loser_seed'),
    col('loser_name'),
    col('loser_hand'),
    col('loser_ioc'),
    col('loser_age'),
    col('score'),
    col('best_of'),
    col('round'),
    col('minutes'),
    col('w_ace'),
    col('w_df'),
    col('w_svpt'),
    col('w_1stIn'),
    col('w_1stWon'),
    col('w_2ndWon'),
    col('w_SvGms'),
    col('w_bpSaved'),
    col('w_bpFaced'),
    col('l_ace'),
    col('l_df'),
    col('l_svpt'),
    col('l_1stIn'),
    col('l_1stWon'),
    col('l_2ndWon'),
    col('l_SvGms'),
    col('l_bpSaved'),
    col('l_bpFaced'),
    col('winner_rank'),
    col('winner_rank_points'),
    col('loser_rank'),
    col('loser_rank_points')
    ).where((col('winner_id') == 207989) | (col('loser_id') == 207989)).withColumn('match_id', monotonically_increasing_id()).cache()

24/06/25 08:35:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
alcaraz_matches_over_the_years = alcaraz_df.groupby(col('tourney_date')[0:4].alias('Year')).count().orderBy(col('Year')).withColumnRenamed('count', 'Matches')
alcaraz_wins_over_the_years = alcaraz_df.where(col('winner_id') == '207989').groupby(col('tourney_date')[0:4].alias('Year')).count().orderBy(col('Year')).withColumnRenamed('count', 'Wins')
alcaraz_losses_over_the_years = alcaraz_df.where(col('loser_id') == '207989').groupby(col('tourney_date')[0:4].alias('Year')).count().orderBy(col('Year')).withColumnRenamed('count', 'Losses')

alcaraz_matches_wins_losses_over_the_years = alcaraz_matches_over_the_years.join(alcaraz_wins_over_the_years,['Year'])
alcaraz_matches_wins_losses_over_the_years = alcaraz_matches_wins_losses_over_the_years.join(alcaraz_losses_over_the_years,['Year']).orderBy(col('Year'))

alcaraz_matches_wins_losses_percentage_over_the_years = alcaraz_matches_wins_losses_over_the_years.withColumn('%', (col('Wins')/ ((col('Wins')+col('Losses'))) * 100).cast('int') )
alcaraz_matches_wins_losses_percentage_over_the_years.show()

+----+-------+----+------+---+
|Year|Matches|Wins|Losses|  %|
+----+-------+----+------+---+
|2020|      2|   1|     1| 50|
|2021|     49|  32|    17| 65|
|2022|     70|  57|    13| 81|
|2023|     77|  65|    12| 84|
|2024|     23|  18|     5| 78|
+----+-------+----+------+---+



In [7]:
alcaraz_matches_each_surface = alcaraz_df.groupby(col('Surface')).count().withColumnRenamed('count', 'Matches')
alcaraz_wins_each_surface = alcaraz_df.where(col('winner_id') == '207989').groupby(col('Surface')).count().withColumnRenamed('count', 'Wins')
alcaraz_losses_each_surface = alcaraz_df.where(col('loser_id') == '207989').groupby(col('Surface')).count().withColumnRenamed('count', 'Losses')

alcaraz_matches_wins_losses_each_surface = alcaraz_matches_each_surface.join(alcaraz_wins_each_surface,['Surface'])
alcaraz_matches_wins_losses_each_surface = alcaraz_matches_wins_losses_each_surface.join(alcaraz_losses_each_surface,['Surface'])

alcaraz_matches_wins_losses_percentage_each_surface = alcaraz_matches_wins_losses_each_surface.withColumn('%', (col('Wins')/ ((col('Wins')+col('Losses'))) * 100).cast('int') )

alcaraz_matches_wins_losses_percentage_each_surface.show()

+-------+-------+----+------+---+
|Surface|Matches|Wins|Losses|  %|
+-------+-------+----+------+---+
|   Clay|     86|  69|    17| 80|
|   Hard|    117|  88|    29| 75|
|  Grass|     18|  16|     2| 88|
+-------+-------+----+------+---+



In [8]:
alcaraz_titles_by_level = alcaraz_df.where((col('winner_id') == '207989') & (col('round') == 'F')).groupBy(col('tourney_level').alias('Tourney Level')).count()
alcaraz_titles_by_level = alcaraz_titles_by_level.select(when(col('Tourney Level') == 'M', 'Masters').when(col('Tourney Level') == 'A', 'ATP').when(col('Tourney Level') == 'G', 'Grand Slams').alias('Level'),col('count').alias('Titles'))
alcaraz_titles_by_level.show()

+-----------+------+
|      Level|Titles|
+-----------+------+
|    Masters|     5|
|        ATP|     7|
|Grand Slams|     2|
+-----------+------+



In [9]:
alcaraz_titles_by_name_level_year = alcaraz_df.select(col('tourney_name').alias('Tourney Name'),when(col('tourney_level') == 'M', 'Masters').when(col('tourney_level') == 'A', 'ATP').when(col('tourney_level') == 'G', 'Grand Slams').alias('Level'), col('tourney_date')[0:4].alias('Year')).where((col('winner_id') == '207989') & (col('round') == 'F')).orderBy(col('Year'), col('Level'))
alcaraz_titles_by_name_level_year.show()

+--------------------+-----------+----+
|        Tourney Name|      Level|Year|
+--------------------+-----------+----+
|                Umag|        ATP|2021|
|      NextGen Finals|        ATP|2021|
|      Rio de Janeiro|        ATP|2022|
|           Barcelona|        ATP|2022|
|             Us Open|Grand Slams|2022|
|       Miami Masters|    Masters|2022|
|      Madrid Masters|    Masters|2022|
|        Buenos Aires|        ATP|2023|
|           Barcelona|        ATP|2023|
|        Queen's Club|        ATP|2023|
|           Wimbledon|Grand Slams|2023|
|Indian Wells Masters|    Masters|2023|
|      Madrid Masters|    Masters|2023|
|Indian Wells Masters|    Masters|2024|
+--------------------+-----------+----+



In [10]:
alcaraz_rank_progression = alcaraz_df.select(col('tourney_date')[0:6].alias('Date'), col('winner_rank').alias('Rank')).distinct().where(col('winner_id') == 207989).orderBy(col('Date'))
alcaraz_rank_progression.show()

+------+----+
|  Date|Rank|
+------+----+
|202002| 406|
|202102| 141|
|202102| 146|
|202104| 133|
|202105| 120|
|202105|  97|
|202106|  75|
|202107|  73|
|202108|  55|
|202108|  54|
|202110|  42|
|202111|  35|
|202111|  32|
|202201|  31|
|202202|  29|
|202203|  16|
|202203|  19|
|202204|  11|
|202205|   6|
|202205|   9|
+------+----+
only showing top 20 rows



In [11]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(alcaraz_df) for column in ["winner_name"]] # "tourney_level", "surface", "tourney_name",
pipeline = Pipeline(stages=indexers)
indexed_df = pipeline.fit(alcaraz_df).transform(alcaraz_df)

feature_columns = ["winner_name_index"] # "tourney_level_index", "surface_index", "tourney_name_index",
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_df = assembler.transform(indexed_df)

onehot = OneHotEncoder(inputCols=['winner_name_index'], # 'tourney_level_index', 'surface_index', 'tourney_name_index',
                        outputCols=['winner_name_dummy']) # 'tourney_level_dummy', 'surface_dummy', 'tourney_name_dummy',

onehot = onehot.fit(assembled_df)
alcaraz_onehot = onehot.transform(assembled_df)
alcaraz_onehot.select('winner_name_index', 'winner_name','features', 'winner_name_dummy').distinct().sort('winner_name_index').show(5)

(training_data, test_data) = alcaraz_onehot.randomSplit([0.8, 0.2], seed=1234)

+-----------------+--------------------+--------+-----------------+
|winner_name_index|         winner_name|features|winner_name_dummy|
+-----------------+--------------------+--------+-----------------+
|              0.0|      Carlos Alcaraz|   [0.0]|   (30,[0],[1.0])|
|              1.0|    Alexander Zverev|   [1.0]|   (30,[1],[1.0])|
|              2.0|       Jannik Sinner|   [2.0]|   (30,[2],[1.0])|
|              3.0|Felix Auger Alias...|   [3.0]|   (30,[3],[1.0])|
|              4.0|      Novak Djokovic|   [4.0]|   (30,[4],[1.0])|
+-----------------+--------------------+--------+-----------------+
only showing top 5 rows



In [12]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

regression = LinearRegression(labelCol='winner_name_index').fit(training_data)
predictions = regression.transform(test_data)
predictions.select('winner_name', 'loser_name','winner_name_index', 'prediction').show(10, False)

RegressionEvaluator(labelCol='winner_name_index').evaluate(predictions)

24/06/25 08:36:13 WARN Instrumentation: [2cea7c5e] regParam is zero, which might cause numerical instability and overfitting.
24/06/25 08:36:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/25 08:36:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+--------------+---------------------------+-----------------+---------------------+
|winner_name   |loser_name                 |winner_name_index|prediction           |
+--------------+---------------------------+-----------------+---------------------+
|David Goffin  |Carlos Alcaraz             |15.0             |14.999999999999996   |
|Carlos Alcaraz|Nuno Borges                |0.0              |3.675229437492464E-16|
|Carlos Alcaraz|Alejandro Davidovich Fokina|0.0              |3.675229437492464E-16|
|Carlos Alcaraz|Jack Draper                |0.0              |3.675229437492464E-16|
|Carlos Alcaraz|Casper Ruud                |0.0              |3.675229437492464E-16|
|Tommy Paul    |Carlos Alcaraz             |11.0             |10.999999999999998   |
|Carlos Alcaraz|Mackenzie Mcdonald         |0.0              |3.675229437492464E-16|
|Carlos Alcaraz|Jordan Thompson            |0.0              |3.675229437492464E-16|
|Carlos Alcaraz|Tommy Paul                 |0.0              |3.6

2.207061533827349e-15

In [15]:
print(regression.coefficients)
print(regression.coefficients[0])
print(regression.intercept)

[0.9999999999999998]
0.9999999999999998
3.675229437492464e-16
